In [23]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pprint import pprint
from IPython.display import display
from tqdm import tqdm
import glob

%load_ext autoreload
%autoreload 2
%matplotlib inline


sys.path.append(os.path.dirname(os.getcwd()))
from src import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
from dotenv import load_dotenv

load_dotenv()

True

In [25]:
# use the already-imported src.utils (imported in an earlier cell)
from src.config import load_config

config = load_config()
print(config)

paths=PathsConfig(output_dir='output/', prompt_dir='src/graph/prompts/') runner=RunnerConfig(intent_node=NodeConfig(model_name='gpt41mini', prompt_cache_key='intent_node'), supervisor_node=NodeConfig(model_name='gpt41mini', prompt_cache_key='supervisor_node'), feedback_node=NodeConfig(model_name='gpt41mini', prompt_cache_key='feedback_node'), goal_node=NodeConfig(model_name='gpt41mini', prompt_cache_key='goal_node'), task_node=NodeConfig(model_name='gpt41mini', prompt_cache_key='task_node')) skills=[RobotSkillConfig(name='robot1', skills=['GoToObject', 'OpenObject', 'CloseObject', 'PickObject', 'PlaceObject'])] tasks={'GoToObject': {'description': 'Move to the specified object.', 'template': 'GoToObject <robot><object>'}, 'OpenObject': {'description': 'Open the specified object.', 'template': 'OpenObject <robot><object>'}, 'CloseObject': {'description': 'Close the specified object.', 'template': 'CloseObject <robot><object>'}, 'PickObject': {'description': 'Pick up the specified object

In [26]:
from src.runner.state import StateMaker


state_maker = StateMaker(config)
state = state_maker.make(user_query="put a fork on the island table")

pprint(state)

Making inputs for state...
Group: counter_1_left_left_group
Group: island_left_group
url: http://127.0.0.1:8800
Groups found: dict_keys(['counter_corner_main_main_group', 'counter_main_main_group', 'stovetop_main_group', 'counter_1_right_main_group', 'fridge_main_group', 'fridge_housing_main_group', 'stack_1_main_group', 'stack_2_main_group', 'stack_3_main_group', 'hood_main_group', 'cab_main_main_group', 'shelves_main_group', 'fridge_cab_main_group', 'cab_1_left_group', 'window_group', 'cab_2_left_group', 'cab_corner_3_left_group', 'cab_corner_4_left_group', 'sink_left_group', 'counter_1_left_left_group', 'counter_corner_left_group', 'island_left_group', 'microwave_left_group', 'stack_1_left_group', 'stack_2_left_group', 'stack_3_left_group'])
{'feedback_result': {},
 'inputs': {'group_list_text': '[\n'
                               '    "counter_corner_main_main_group",\n'
                               '    "counter_main_main_group",\n'
                               '    "stovetop

In [32]:
import time
from src.runner.runner import SupervisedPlanRunner

runner = SupervisedPlanRunner(config=config)

# Add a small delay if you're making multiple rapid requests
# time.sleep(1)

final_state = runner.invoke(state)

2025-12-05 13:40:49,874 [INFO] src.runner.graph: ============= USER_INPUT_NODE ==============
2025-12-05 13:40:51,117 [INFO] src.runner.graph: User Query: place a apple on island.

2025-12-05 13:40:51,121 [INFO] src.runner.graph: ============= INTENT_NODE ==============
2025-12-05 13:40:52,863 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-05 13:40:52,877 [INFO] src.runner.graph: AI Answer:
{'intent': 'new'}

2025-12-05 13:40:52,882 [INFO] src.runner.graph: ============= SUPERVISOR_NODE ==============
2025-12-05 13:40:54,905 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-05 13:40:54,911 [INFO] src.runner.graph: AI Answer:
{'is_feasible': False, 'reasons': ['사과 객체가 존재하지 않음', '사과가 위치한 그룹이 없음'], 'user_final_query': '사과를 아일랜드 식탁에 놓아줘.'}

2025-12-05 13:40:54,914 [INFO] src.runner.graph: ============= FEEDBACK_NODE ==============
2025-12-05 13:40:57,029 [INFO] httpx: HTTP Request: PO

In [39]:
graph = runner.graph
print(graph)

In [ ]:
# import time
# from src.runner.runner import PlanRunner

# runner = PlanRunner(config=config)

# # Add a small delay if you're making multiple rapid requests
# # time.sleep(1)

# final_state = runner.invoke(state)

2025-12-04 21:19:04,695 [INFO] src.runner.graph: ============= GOAL_NODE ==============
2025-12-04 21:19:06,099 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-04 21:19:06,102 [INFO] src.runner.graph: AI Answer:
{'subgoals': ['Put a fork on the island table']}

2025-12-04 21:19:06,103 [INFO] src.runner.graph: ============= TASK_NODE ==============


Subgoals Text:
1. Put a fork on the island table



2025-12-04 21:19:09,184 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-04 21:19:09,193 [INFO] src.runner.graph: AI Answer:
{'tasks': [{'subgoal': 'Put a fork on the island table', 'tasks': [{'skill': 'GoToObject', 'target': 'object_fork_0'}, {'skill': 'PickObject', 'target': 'object_fork_0'}, {'skill': 'GoToObject', 'target': 'island_left_group'}, {'skill': 'PlaceObject', 'target': 'island_left_group'}]}]}



In [42]:
graph_image = graph.get_graph().draw_mermaid_png()
with open("graph.png", "wb") as f:
    f.write(graph_image)